In [1]:
cd /home/ubuntu/fedatk_unl_tj/

/home/ubuntu/fedatk_unl_tj


In [ ]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd
import gc

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from utils.util_notebooks import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *

/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
setting, num_user = "FedAvg_adv", 40
exp = "cifar10"

try: # Skip loading if already loaded
    aggregator
except:
    aggregator, clients, args_ = set_args(setting, num_user,  experiment = exp) # Indicate dataset here

# Load models for FAT and FedAvg
save_path_FAT = '/home/ubuntu/fedatk_unl_tj/weights/cifar10/231031_FAT150round/FAT/'
save_path_FedAvg =  '/home/ubuntu/fedatk_unl_tj/weights/cifar10/230922_baseline_train/fedavg/'

model_FAT = copy.deepcopy(import_model_weights(num_user, setting, save_path_FAT, aggregator, args_)[0])
model_Fedavg = import_model_weights(num_user, setting, save_path_FedAvg, aggregator, args_)[0]

# del aggregator, clients, args_
# torch.cuda.empty_cache()


==> Clients initialization..
===> Building data iterators..


 49%|████▉     | 39/80 [00:00<00:00, 99.68it/s] 


===> Initializing clients..


  0%|          | 0/40 [00:00<?, ?it/s]/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/anaconda3/envs/FedEM_env/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 40/40 [00:17<00:00,  2.28it/s]
/home/ubuntu/fedatk_unl_tj/aggregator.py:289: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious p

In [4]:
# Obtain parameters for each layer
params_FAT = model_FAT.state_dict()
params_FedAvg = model_Fedavg.state_dict()

# Just take the values of weights and bias for the model
desired_keys = [key for key in params_FAT.keys() if 'weight' in key or 'bias' in key]

In [7]:
num_aru = [5, 10 ,15]
beta_params = [0.05, 0.05, 0.05]
weight2_list = [1 ,1, 1]
layer_threshold = [150, 50, 50]
num_rounds = 10
eps_train = 4.5
eps_attack = 4

agg_options = ["median_sublayers", "median_sublayers", "median_sublayers"]
# agg_options = ['median', 'median', 'median']

result_list = []
for itt in range(len(num_aru)):
    result_list += [{}]

# setting, num_user = "FedAvg_adv", 20
save_path_FedAvg_150R = save_path_FAT
# aggregator, clients, args_ = set_args(setting, num_user,  experiment = exp)
torch.cuda.empty_cache()

agg_choice = random.choices(range(3),k=num_rounds)

if True:
    print("updating adv data set")

    print(f"  Memory allocated - before adv: {torch.cuda.memory_allocated() / 1e6:.2f} MB")

    # add adv dataset update
    Fu = np.ones(num_user) * 0.4
    # Fu[0:10] = 1

    # Setting evasion attack parameters
    x_min = torch.min(aggregator.clients[0].adv_nn.dataloader.x_data).detach().cuda()
    x_max = torch.max(aggregator.clients[0].adv_nn.dataloader.x_data).detach().cuda()
    atk_params = PGD_Params()
    atk_params.set_params(batch_size=1, iteration = 10,
                    target = -1, x_val_min = x_min, x_val_max = x_max,
                    step_size = 0.05, step_norm = "inf", eps = eps_train, eps_norm = 2)

    # Assign proportion and attack params
    for c in range(len(aggregator.clients)):
        if Fu[c] > 0:
            print (c)
            aggregator.clients[c].set_adv_params(Fu[c], atk_params)
            aggregator.clients[c].update_advnn()

            aggregator.clients[c].assign_advdataset()

            # print(f"  Memory allocated - after adv: {torch.cuda.memory_allocated() / 1e6:.2f} MB")
            aggregator.clients[c].del_advnn()

    
    

updating adv data set
  Memory allocated - before adv: 1067.48 MB


AttributeError: 'NoneType' object has no attribute 'dataloader'

In [8]:
for itt in range(len(beta_params)):
    
    aggregator_temp = aggregator

    # Perform rounds of FAT on FedAvg model
    num_adv = num_aru[itt]
    weight2 = 1 / num_adv * weight2_list[itt]
    adv_id = random.sample(range(10, num_user), num_adv)  # excluding 0-9 as Fu = 1
    beta = beta_params[itt]

    test_acc_gather = []
    adv_acc_gather = []
    test_acc_std = []
    adv_acc_std = []
    cosine_gather_layers = np.zeros([num_rounds, len(desired_keys)])

    aggregator_temp.tm_rate = beta

    # Test performance of aggregator on data
    aggregator_temp.load_state(dir_path=save_path_FedAvg_150R)
    aggregator_temp.update_clients()
    model_FA = pull_model_from_agg(aggregator_temp)
    model_FA.eval()
    acc, adv_acc = get_adv_acc(aggregator_temp, model_FA, eps=eps_attack)

    print("Test acc: ", np.mean(acc), "adv acc: ", np.mean(adv_acc))
    test_acc_gather += [np.mean(acc)]
    adv_acc_gather += [np.mean(adv_acc)]
    test_acc_std += [np.std(acc)]
    adv_acc_std += [np.std(adv_acc)]

    del model_FA, acc, adv_acc

    for i in range(num_rounds):
        # Memory profiling before round starts
        print(f"  Memory allocated - pre agg: {torch.cuda.memory_allocated() / 1e6:.2f} MB")
        torch.cuda.reset_peak_memory_stats()

        # Per round Freq
        aggregator_temp.aggregation_op = agg_options[itt]

        # Perform UNL aggregation
        UNL_mix(aggregator_temp, adv_id, model_inject=model_Fedavg, keys=desired_keys, 
                weight_scale_2=weight2, dump_flag=False, tm_beta=beta, 
                median_threshold=layer_threshold[itt])
        print(f"  Memory allocated - post UNL MIX: {torch.cuda.memory_allocated() / 1e6:.2f} MB")

        model_overfit = pull_model_from_agg(aggregator_temp)
        model_overfit.eval()
        acc, adv_acc = get_adv_acc(aggregator_temp, model_overfit, eps=eps_attack)

        print(f"  Memory allocated - post ADV ACC: {torch.cuda.memory_allocated() / 1e6:.2f} MB")

        print("Round", i, "Test acc: ", np.mean(acc), "adv acc: ", np.mean(adv_acc))
        test_acc_gather += [np.mean(acc)]
        adv_acc_gather += [np.mean(adv_acc)]
        test_acc_std += [np.std(acc)]
        adv_acc_std += [np.std(adv_acc)]

        result_list[itt]['test_acc'] = test_acc_gather
        result_list[itt]['adv_acc'] = adv_acc_gather
        result_list[itt]['test_std'] = test_acc_std
        result_list[itt]['adv_std'] = adv_acc_std
        result_list[itt]['num_clients'] = num_adv
        result_list[itt]['beta'] = beta
        result_list[itt]['weight2'] = weight2_list[itt]

        # Clean up after each round
        del model_overfit
    torch.cuda.empty_cache()
    break



Test acc:  0.8021875008940696 adv acc:  0.35218749977648256
  Memory allocated - pre agg: 1067.48 MB
  Memory allocated - post UNL MIX: 1059.83 MB
  Memory allocated - post ADV ACC: 1078.06 MB
Round 0 Test acc:  0.7987500011920929 adv acc:  0.33687499910593033
  Memory allocated - pre agg: 1059.83 MB
  Memory allocated - post UNL MIX: 1063.73 MB
  Memory allocated - post ADV ACC: 1081.87 MB
Round 1 Test acc:  0.8046875029802323 adv acc:  0.32562500014901163
  Memory allocated - pre agg: 1063.73 MB
  Memory allocated - post UNL MIX: 1065.66 MB
  Memory allocated - post ADV ACC: 1083.89 MB
Round 2 Test acc:  0.798124997317791 adv acc:  0.3168750017881393
  Memory allocated - pre agg: 1065.66 MB
  Memory allocated - post UNL MIX: 1066.22 MB
  Memory allocated - post ADV ACC: 1085.73 MB
Round 3 Test acc:  0.7906250059604645 adv acc:  0.28968749940395355
  Memory allocated - pre agg: 1066.22 MB
  Memory allocated - post UNL MIX: 1064.68 MB
  Memory allocated - post ADV ACC: 1082.82 MB
Round